In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder ## 라벨인코딩을 위해 사용
from sklearn.preprocessing import RobustScaler # 단순선형회귀분석을 위해 사용
from sklearn.ensemble import RandomForestRegressor ## 랜덤포레스트 회귀모델
from xgboost import XGBRegressor ## XGBoost 회귀모델
from catboost import CatBoostRegressor ## CatBoost 회귀모델
import lightgbm as LGB ## LGBM 모델
from sklearn.metrics import mean_squared_error ## 평가지표로 사용할 MSE
from sklearn.impute import SimpleImputer ## 결측값 처리 패키지
import statsmodels.api as sm ## 단순선형회귀분석에 사용
from sklearn.model_selection import GridSearchCV
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings("ignore")

path = "D:/git_project/ECO_Jeju/Sungmin/new_datas/final_edition" # waste, korean, long, short, res, card 경로
path2 = "D:/git_project/ECO_Jeju/Sungmin/new_datas/external" # rfid 경로
path3 = "D:/빅콘테스트/데이터/02_평가데이터_update(210806)"


waste = pd.read_csv(path + "/waste_group.csv") 
korean = pd.read_csv(path + "/korean_group.csv")
long = pd.read_csv(path + "/long_group.csv")
short = pd.read_csv(path + "/short_group.csv")
res = pd.read_csv(path + "/resident_group.csv")
card = pd.read_csv(path + "/card_group.csv")
rfid = pd.read_csv(path + "/rfid_group.csv")

## 라벨인코더는 범주형 변수를 명목형 변수(수치형)으로 변환 할때 사용
le = LabelEncoder()
rs = RobustScaler()

## waste 전처리

In [2]:
#%% waste 전처리
waste
waste.info()

new_waste = waste

sample_index = waste["emd_nm"].unique()


emd_nm = new_waste["emd_nm"]
new_waste = new_waste.set_index(["base_date", "emd_nm"])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1666 entries, 0 to 1665
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   base_date      1666 non-null   object
 1   emd_nm         1666 non-null   object
 2   waste_em_cnt   1666 non-null   int64 
 3   waste_em_g     1666 non-null   int64 
 4   waste_pay_amt  1666 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 65.2+ KB


## korean 전처리
### 추자면, 우도면 평가데이터에 없으므로 삭제

In [3]:
#%% korean 전처리

korean.drop(korean.loc[korean["emd_nm"] == "추자면"].index, inplace = True)
korean.drop(korean.loc[korean["emd_nm"] == "우도면"].index, inplace = True)
korean.info()

new_korean = korean

new_korean = new_korean.drop(['korean_sex_남성', 'korean_sex_여성', 'korean_age_0', 'korean_age_10',
       'korean_age_20', 'korean_age_30', 'korean_age_40', 'korean_age_50',
       'korean_age_60', 'korean_age_70', 'korean_age_80'], axis = 1)


new_korean = new_korean.set_index(["base_date", "emd_nm"])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1722 entries, 0 to 1805
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   base_date             1722 non-null   object 
 1   emd_nm                1722 non-null   object 
 2   korean_resd_pop_cnt   1722 non-null   float64
 3   korean_work_pop_cnt   1722 non-null   float64
 4   korean_visit_pop_cnt  1722 non-null   float64
 5   korean_sex_남성         1722 non-null   float64
 6   korean_sex_여성         1722 non-null   float64
 7   korean_age_0          1722 non-null   float64
 8   korean_age_10         1722 non-null   float64
 9   korean_age_20         1722 non-null   float64
 10  korean_age_30         1722 non-null   float64
 11  korean_age_40         1722 non-null   float64
 12  korean_age_50         1722 non-null   float64
 13  korean_age_60         1722 non-null   float64
 14  korean_age_70         1722 non-null   float64
 15  korean_age_80        

## long 전처리

In [4]:
#%% long 전처리 new_long

long
long.info()

long.drop(long.loc[long["emd_nm"] == "추자면"].index, inplace = True)
long.drop(long.loc[long["emd_nm"] == "우도면"].index, inplace = True)


new_long = long

new_long.columns
long["emd_nm"].value_counts()  ## 추자면, 우도면 평가데이터에 없음

new_long = new_long.drop(['long_nationality_AUS', 'long_nationality_BGD',
       'long_nationality_CAN', 'long_nationality_CHN', 'long_nationality_DEU',
       'long_nationality_EGY', 'long_nationality_ETC', 'long_nationality_FRA',
       'long_nationality_GBR', 'long_nationality_IDN', 'long_nationality_IND',
       'long_nationality_JPN', 'long_nationality_KAZ', 'long_nationality_KGZ',
       'long_nationality_KHM', 'long_nationality_LKA', 'long_nationality_MGL',
       'long_nationality_MMR', 'long_nationality_MYS', 'long_nationality_NGR',
       'long_nationality_NPL', 'long_nationality_NZL', 'long_nationality_PAK',
       'long_nationality_PHL', 'long_nationality_RUS', 'long_nationality_THA',
       'long_nationality_TWN', 'long_nationality_UKR', 'long_nationality_USA',
       'long_nationality_UZB', 'long_nationality_VNM'], axis = 1)




new_long = new_long.set_index(["base_date", "emd_nm"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1806 entries, 0 to 1805
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   base_date             1806 non-null   object 
 1   emd_nm                1806 non-null   object 
 2   long_resd_pop_cnt     1806 non-null   float64
 3   long_work_pop_cnt     1806 non-null   float64
 4   long_visit_pop_cnt    1806 non-null   float64
 5   long_nationality_AUS  1806 non-null   int64  
 6   long_nationality_BGD  1806 non-null   int64  
 7   long_nationality_CAN  1806 non-null   int64  
 8   long_nationality_CHN  1806 non-null   int64  
 9   long_nationality_DEU  1806 non-null   int64  
 10  long_nationality_EGY  1806 non-null   int64  
 11  long_nationality_ETC  1806 non-null   int64  
 12  long_nationality_FRA  1806 non-null   int64  
 13  long_nationality_GBR  1806 non-null   int64  
 14  long_nationality_IDN  1806 non-null   int64  
 15  long_nationality_IND 

## short 전처리

In [5]:
#%% short 전처리 new_short

short
short.info()
short.columns
short.drop(short.loc[short["emd_nm"] == "추자면"].index, inplace = True)
short.drop(short.loc[short["emd_nm"] == "우도면"].index, inplace = True)

new_short = short
new_short = new_short.drop(['short_nationality_CHN', 'short_nationality_ETC',
       'short_nationality_HKG', 'short_nationality_IDN',
       'short_nationality_JPN', 'short_nationality_MYS',
       'short_nationality_SGP', 'short_nationality_THA',
       'short_nationality_USA', 'short_nationality_VNM'], axis = 1)


new_short = new_short.set_index(["base_date", "emd_nm"])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1806 entries, 0 to 1805
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   base_date              1806 non-null   object 
 1   emd_nm                 1806 non-null   object 
 2   short_nationality_CHN  1806 non-null   int64  
 3   short_nationality_ETC  1806 non-null   int64  
 4   short_nationality_HKG  1806 non-null   int64  
 5   short_nationality_IDN  1806 non-null   int64  
 6   short_nationality_JPN  1806 non-null   int64  
 7   short_nationality_MYS  1806 non-null   int64  
 8   short_nationality_SGP  1806 non-null   int64  
 9   short_nationality_THA  1806 non-null   int64  
 10  short_nationality_USA  1806 non-null   int64  
 11  short_nationality_VNM  1806 non-null   int64  
 12  short_visit_pop_cnt    1806 non-null   float64
dtypes: float64(1), int64(10), object(2)
memory usage: 183.5+ KB


## resident 전처리

In [6]:
#%% resdient 

res
res.info()
res.isna().sum()
res.drop(res.loc[res["emd_nm"] == "추자면"].index, inplace = True)
res.drop(res.loc[res["emd_nm"] == "우도면"].index, inplace = True)

new_res = res


new_res = new_res.set_index(["base_date", "emd_nm"])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1806 entries, 0 to 1805
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   base_date               1806 non-null   object
 1   emd_nm                  1806 non-null   object
 2   resident_resid_reg_pop  1806 non-null   int64 
 3   resident_foreign_pop    1806 non-null   int64 
 4   resident_total_pop      1806 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 70.7+ KB


## card 전처리

In [8]:
#%% card 전처리

card
card.info()
card.isna().sum()

card["emd_nm"].value_counts()
card.drop(card.loc[card["emd_nm"] == "추자면"].index, inplace = True)
card.drop(card.loc[card["emd_nm"] == "우도면"].index, inplace = True)

new_card = card


new_card = new_card.set_index(["base_date", "emd_nm"])




<class 'pandas.core.frame.DataFrame'>
Int64Index: 1764 entries, 0 to 1847
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   base_date      1764 non-null   object
 1   emd_nm         1764 non-null   int32 
 2   use_cnt        1764 non-null   int64 
 3   mct_cat_nm_0   1764 non-null   int64 
 4   mct_cat_nm_1   1764 non-null   int64 
 5   mct_cat_nm_2   1764 non-null   int64 
 6   mct_cat_nm_3   1764 non-null   int64 
 7   mct_cat_nm_4   1764 non-null   int64 
 8   mct_cat_nm_5   1764 non-null   int64 
 9   mct_cat_nm_6   1764 non-null   int64 
 10  mct_cat_nm_7   1764 non-null   int64 
 11  mct_cat_nm_8   1764 non-null   int64 
 12  mct_cat_nm_9   1764 non-null   int64 
 13  mct_cat_nm_10  1764 non-null   int64 
dtypes: int32(1), int64(12), object(1)
memory usage: 199.8+ KB


## rifd 전처리

In [9]:
#%% rifd 전처리

rfid.info()
rfid.isna().sum()

new_rfid = rfid


new_rfid = new_rfid.set_index(["base_date", "emd_nm"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1722 entries, 0 to 1721
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   base_date    1722 non-null   object 
 1   emd_nm       1722 non-null   object 
 2   disQuantity  1722 non-null   float64
 3   disCount     1722 non-null   float64
dtypes: float64(2), object(2)
memory usage: 53.9+ KB


## 데이터 합치기 및 train, test set 분리
### 각 년도의 7, 8월 데이터의 평균으로 2021년 7,8월 데이터 생성


In [10]:
#%% 데이터 합치기, train test 셋 분리
data = pd.concat([new_waste, new_korean, new_long, new_short, new_res, new_card, new_rfid], axis = 1)
col = data.columns
data.reset_index(inplace=True)

data.set_index(["base_date","emd_nm"], inplace=True)
data_index = data.index

# 7월 평균 데이터 생성
sum_seven = data.loc['2018-07']+data.loc['2019-07']+data.loc['2020-07']
avg_seven = sum_seven/3
# 8월 평균데이터 생성
sum_eight = data.loc['2018-08']+data.loc['2018-08']+data.loc['2018-08']
avg_eight = sum_eight/3

# 멀티인덱스 생성
index_2 = [['2021-07','2021-08'],['건입동','구좌읍','남원읍','노형동','대륜동','대정읍','대천동','도두동','동홍동','봉개동','삼도1동','삼도2동','삼양동','서홍동','성산읍','송산동','아라동','안덕면','알수없음','애월읍','연동','영천동',
                                  '예래동','오라동','외도동','용담1동','용담2동','이도1동','이도2동','이호동','일도1동','일도2동','정방동','조천읍','중문동','중앙동','천지동','표선면','한경면','한림읍','화북동','효돈동']]


multi_index_2 = pd.MultiIndex.from_product(index_2)
new_index = data_index.append(multi_index_2)



new_data = pd.concat([data, avg_seven, avg_eight])
new_data.index = new_index

## 결측값을 각 피처들의 평균값으로 대체하기 위해 SimpleImputer 사용


new_data.reset_index(inplace=True)

le.fit(new_data["emd_nm"])
emd_nm = le.transform(new_data["emd_nm"])
new_data["emd_nm"] = emd_nm
new_data.set_index(["base_date", "emd_nm"], inplace=True)

imputer = SimpleImputer(strategy = 'mean')
imp_data = pd.DataFrame(imputer.fit_transform(new_data))
imp_data.columns = col
imp_data = imp_data.set_index(new_index)

imp_data.info()


## 후진 제거법을 위한 단순선형회귀분석

In [11]:
train_rs_x = rs.fit_transform(train_x)
model1 = sm.OLS(train_y, train_rs_x) 
fitted_model = model1.fit()
fitted_model.summary() 
y_pred = fitted_model.predict(test_x)
scores = mean_squared_error(test_y, y_pred)**0.5


## 다중공선성 확인을 위한 VIF 검증

In [12]:
features = "waste_em_cnt+short_visit_pop_cnt+resident_resid_reg_pop+resident_foreign_pop+resident_total_pop+mct_cat_nm_2+mct_cat_nm_3+mct_cat_nm_5+mct_cat_nm_10+disQuantity"
w, z = dmatrices("waste_em_g ~" + features, data=imp_data, return_type = "dataframe")
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(z.values, i) for i in range(z.shape[1])]
vif["features"] = z.columns

## 모델링
* Randomforest regressor, XGBoost, CatBoostRegressor, Lightgbm 사용
* 각 분석모델의 기본 모델링을 진행 한 후 하이퍼 파라미터 튜닝 후에 RMSE 비교

In [16]:
#%% 모델링
models = []
models.append(['Random Forest', RandomForestRegressor()])
models.append(['XGBoost', XGBRegressor()])
models.append(['CatBoostRegressor', CatBoostRegressor(logging_level=("Silent"))])
models.append(['Lightgbm', LGB.LGBMRegressor()])


list_1 = []


for m in range(len(models)):
    print(models[m])
    model = models[m][1]
    model.fit(train_x, train_y)
    y_pred = model.predict(test_x)
    scores = mean_squared_error(test_y, y_pred)**0.5
    list_1.append(scores)

df_1 = pd.DataFrame(models)    
df = pd.DataFrame(list_1)
df.index = df_1.iloc[:,0]

df

['Random Forest', RandomForestRegressor()]
['XGBoost', XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=None)]
['CatBoostRegressor', <catboost.core.CatBoostRegressor object at 0x000001CDAE209340>]
['Lightgbm', LGBMRegressor()]


,0
0,
Random Forest,3.653529e+06
XGBoost,3.431208e+06
CatBoostRegressor,2.947498e+06
Lightgbm,4.602344e+06


## LGBM 모델 하이퍼 파라미터 튜닝 및 모델링

In [24]:
gridParams = { 
    'learning_rate': [0.005],
    'n_estimators': [40],
    'num_leaves': [16,32, 64], 
    'random_state' : [501],
    'num_boost_round' : [3000],
    'colsample_bytree' : [0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4], 
    }

lgbm  = LGB.LGBMRegressor(n_estimators=100)

gridcv = GridSearchCV(lgbm, param_grid = gridParams, cv = 3)
gridcv.fit(train_x, train_y, eval_metric = 'mse')


print('Optimized hyperparameters', gridcv.best_params_)

[LightGBM] [Warning] num_iterations is set=3000, num_boost_round=3000 will be ignored. Current value: num_iterations=3000
[LightGBM] [Warning] num_iterations is set=3000, num_boost_round=3000 will be ignored. Current value: num_iterations=3000
[LightGBM] [Warning] num_iterations is set=3000, num_boost_round=3000 will be ignored. Current value: num_iterations=3000
[LightGBM] [Warning] num_iterations is set=3000, num_boost_round=3000 will be ignored. Current value: num_iterations=3000
[LightGBM] [Warning] num_iterations is set=3000, num_boost_round=3000 will be ignored. Current value: num_iterations=3000
[LightGBM] [Warning] num_iterations is set=3000, num_boost_round=3000 will be ignored. Current value: num_iterations=3000
[LightGBM] [Warning] num_iterations is set=3000, num_boost_round=3000 will be ignored. Current value: num_iterations=3000


KeyboardInterrupt: 

In [18]:
#%%
model = LGB.LGBMRegressor(colsample_bytree = 0.65,
                           learning_rate = 0.005,
                           n_estimators = 40,
                           num_iterations=3000,
                           feature_fraction=0.7,
                           num_leaves = 63,
                           random_state = 501,
                           reg_alpha = 1,
                           reg_lambda = 1, 
                           subsample = 0.7)

model.fit(train_x, train_y)
y_pred = model.predict(test_x)

scores = mean_squared_error(test_y, y_pred)**0.5

y_pred
print(scores)

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=0.65 will be ignored. Current value: feature_fraction=0.7


4352473.9019963145

## CatBoost Regressor 모델링

In [15]:
# CatBoost
model2 = CatBoostRegressor(logging_level = ("Silent"))

model2.fit(train_x, train_y)
y_pred2 = model2.predict(test_x)

scores2 = mean_squared_error(test_y, y_pred2) ** 0.5

y_pred2
print(scores2)

2947498.404632697

## RandomForest Regressor 하이퍼 파라미터 튜닝 및 모델링

In [27]:
gridParams2 = {'n_estimators' : [10,100],
          'max_depth' : [6,8,10,12],
          'min_samples_leaf' : [8, 12, 18],
          'min_samples_split' : [8, 16, 20]
          }
rf  = RandomForestRegressor(n_estimators=100)

gridcv = GridSearchCV(rf, param_grid = gridParams2, cv = 3)
gridcv.fit(train_x, train_y)


print('Optimized hyperparameters', gridcv.best_params_) 

KeyboardInterrupt: 

In [20]:
#%% 랜덤포레스트 모델선정 6286393.022425565
model3 = RandomForestRegressor(max_depth = 10,
                           min_samples_leaf = 8,
                           n_estimators = 100,
                           min_samples_split = 8,
                           )

model3.fit(train_x, train_y)
y_pred3 = model3.predict(test_x)

scores3 = mean_squared_error(test_y, y_pred3)**0.5

y_pred3
print(scores3)


5921443.16524095


## Xgb Regressor 하이퍼파라미터 튜닝 및 모델링

## 파일 제출
### 하나의 파일 형식을 만들어서 붙여넣기로 제출함

In [21]:
#%% Xgboost regressor 하이퍼 파라미터튜닝
gridParams3 = {'n_estimators' : [40,400,4000],
          'max_depth' : [6,8,10,12],
          'colsample_bytree' : [0.2, 0.9, 0.1],
          'eta' : [0.01, 0.05, 0.1, 0.5]
          }
reg  = XGBRegressor()

gridcv = GridSearchCV(reg, param_grid = gridParams3, cv = 3)
gridcv.fit(train_x, train_y, eval_metric = "rmse")


print('Optimized hyperparameters', gridcv.best_params_) 


KeyboardInterrupt: 

In [22]:
#%% Xgb regressor 모델 선정

model4 = XGBRegressor(n_estimators = 4000,
                      max_depth = 6,
                      colsample_bytree = 0.9,
                      eta = 0.1,)

model4.fit(train_x, train_y)
y_pred4 = model4.predict(test_x)

scores4 = mean_squared_error(test_y, y_pred4) ** 0.5

y_pred4
print(scores4)

3583540.009599851


* CatBoost Regressor의 rmse값이 가장 낮으므로 CatBoost Regressor를 채택

In [23]:
seven = np.around(y_pred2[0:42],1) #7월
eight = np.around(y_pred2[42:],1) #8월

DF = pd.DataFrame({'행정동명': sample_index, '7월 배출량(g)': seven, '8월 배출량(g)': eight})

submission = pd.read_csv(path3 + "/sample_submission2.csv")
submission.set_index(submission["NO"], inplace = True)
submission.drop(["NO"], axis = 1, inplace=True)

submission = pd.merge(submission, DF, on = '행정동명', how = 'inner')
submission.drop(["7월 배출량(g)_x", "8월 배출량(g)_x"], axis = 1, inplace = True)
submission.columns = ['행정동명', '7월 배출량(g)', '8월 배출량(g)']
submission.to_csv('D:/빅콘테스트/데이터/02_평가데이터_update(210806)/result.csv')
